###        To Do.
- Currently the alias_pairs function is matching the exact keywords ie. for **ahu_enable_sensor** and **ahu_chiller_enable_sensor** we have ```ahu_1_enable_sensor``` and ``ahu_1_chiller_enable_sensor``, respectively. These aliases would match because both contain the same key "1". But if the config provided kewyords are changed like **ahu_enable_sensor** and **ahu_chiller** then the available aliases would not match. So your task is to update alias_pair function accordingly.
    
- Make sure all KPIs do not throw any error.

- There are some bugs in the code, so fix that before you start your main task.
    

In [5]:
import arrow
import pandas as pd
import json
import google.auth
import google.auth
import logging
from string import Template
from meteostat import Daily
from google.cloud import bigquery
import datetime
import pandas as pd
import numpy as np
import regex as re
from datetime import time
from functools import reduce



import warnings
warnings.filterwarnings('ignore')

In [6]:
df_main=pd.read_csv('D:\\Cloud-Track\\assign1\\main_data.csv')
df_main

,time_local,alias,value
0,2022-12-23 01:00:00,sp_zenith,150.928824
1,2022-12-23 01:00:00,lat,50.875698
2,2022-12-23 01:00:00,sp_apparent_zenith,150.928824
3,2022-12-23 01:00:00,dew_point,2.971764
4,2022-12-23 01:00:00,dhi,0.000000
...,...,...,...
1344757,2022-09-30 02:00:00,sp_azimuth,19.193106
1344758,2022-09-30 02:00:00,precipitation,0.000000
1344759,2022-09-30 02:00:00,cloud_cover,91.600000
1344760,2022-09-30 02:00:00,elevation,194.000000


In [8]:
df_json = pd.read_json('D:\\Cloud-Track\\assign1\\kpi_config.json')
df_json

,_comment_,settings,rules,logs_link
mute_rules,This is KPI settings,0.0,NaN,https://colab.research.google.com/drive/16lMaX...
station_id,This is KPI settings,10567.0,NaN,https://colab.research.google.com/drive/16lMaX...
kpi_name,This is KPI settings,NaN,"{'1': 'run_time_hours', '2': 'run_time_fractio...",https://colab.research.google.com/drive/16lMaX...
alias_a,This is KPI settings,NaN,"{'1': 'enable_cmd', '2': 'enable_cmd', '3': 'a...",https://colab.research.google.com/drive/16lMaX...
alias_b,This is KPI settings,NaN,"{'1': None, '2': None, '3': 'ahu_fan_air_enabl...",https://colab.research.google.com/drive/16lMaX...
alias_c,This is KPI settings,NaN,"{'1': None, '2': None, '3': None, '4': 'heatEx...",https://colab.research.google.com/drive/16lMaX...
alias_d,This is KPI settings,NaN,"{'1': None, '2': None, '3': None, '4': None, '...",https://colab.research.google.com/drive/16lMaX...
alias_e,This is KPI settings,NaN,"{'1': None, '2': None, '3': None, '4': None, '...",https://colab.research.google.com/drive/16lMaX...
alias_f,This is KPI settings,NaN,"{'1': None, '2': None, '3': None, '4': None, '...",https://colab.research.google.com/drive/16lMaX...
equation_1,This is KPI settings,NaN,"{'1': 'alias_a*24', '2': 'alias_a', '3': '(1 i...",https://colab.research.google.com/drive/16lMaX...


In [137]:
class kpi():
    def __init__(self, rule, batch):
        # get subset of data, min-time and max-time for the given start and end time
        batch, batch_min_time, batch_max_time = self.__batch_subset(batch, rule['start_time'], rule['end_time'])
        # indices of the rule dataframe
        alias_cols = rule.index
        subs = 'alias'
        # get columns of rule book that alias in their names
        self.__res = [alias_col for alias_col in alias_cols if subs in alias_col]
        self.__rule = rule

        # default logs columns
        log_columns = ['kpi_name', 'time', 'aliases', 'value', 'aggregation']
        for col in self.__res:
            log_columns.append(col)

        # create empty dataframe for the logs
        self.__df_logs = pd.DataFrame(columns=['kpi_name', 'time', 'aliases', 'value', 'aggregation'])
        aliases = []
        # get a list of aliases for the given targets in the rule book and append that lists
        # in the aliases list
        for i in self.__res:
            if rule[i] is not None:
                print("column name ==> ", i)
                aliases.append(self.__alias_substrings(batch, [rule[i]], col_name='alias'))
        # in case aliases list length is 1
        # create logs for only one target (alias_a)
        if len(aliases) == 1:
            for alias in aliases[0]:
                # get a dataframe for each alias
                df = self.alias_dataframe(self, data_rd=batch, aliases=alias, kpi=rule['kpi_name'],
                                          resample_grain=None, daily_resample=None, agg=None, non_negative=None)
                # apply the equation that is given in the rule book (equation_1)
                df = self.apply_equation(self, df, equation=rule['equation_1'], temp=True)
                # Aggregate the data after applying equation_1
                df = self.apply_aggregation(self, df, aggregation=rule['aggregation'], kpi=rule['kpi_name'])
                # In case equation_2 is given then apply it after aggregation
                if rule['equation_2'] is not None:
                    df = self.apply_equation(self, df, equation=rule['equation_2'])

                df1 = self.__logs_data(df, rule, [alias])
                self.__df_logs = pd.concat([self.__df_logs, df1], axis=0)

        else:
            alias_pair = self.__alias_pairs(self, batch, rule, aliases)

            for pair in alias_pair:
                if len(pair) != 0:

                    df = self.alias_pairs_dataframe(self, data_rd=batch, aliases=pair, kpi=rule['kpi_name'],
                                                    resample_grain=None, daily_resample=None, agg=None,
                                                    non_negative=None)
                    df = self.apply_equation(self, df, equation=rule['equation_1'], temp=True)
                    df = self.apply_aggregation(self, df, aggregation=rule['aggregation'], kpi=rule['kpi_name'])
                    #print(df)
                    # In case equation_2 is given then apply it after aggregation
                    if rule['equation_2'] is not None:
                        df = self.apply_equation(self, df, equation=rule['equation_2'])
                        #print(df)


                    df1 = self.__logs_data(df, rule, pair)
                    self.__df_logs = pd.concat([self.__df_logs, df1], axis=0)

    @staticmethod
    def __logs_data(df=None, rule=None, alias_list=None):
        """ Gets resultant dataframe and rules for each kpi and creates logs
            in the form of a dataframe.
        Parameters
        ----------
        df: dataframe
        Resultant dataframe for each kpi
        rule:dataframe
        Its the config file
        tgt_alias:string
        Name of the target alias
        ref_alias: string
        Name of the reference alias
        """
        df1 = pd.DataFrame()

        if len(alias_list) == 1:
            logs_alias = alias_list[0]
        else:
            logs_alias = ",".join(alias_list)

        new_row = {'kpi_name': rule['kpi_name'],
                   'time': df['time'],
                   'aliases': logs_alias,
                   'value': df['value'],
                   'equation_1': rule['equation_1'],
                   'equation_2': rule['equation_2'],
                   'aggregation': rule['aggregation'],
                   }

        df1 = pd.DataFrame(new_row)
        return df1

    @staticmethod
    def apply_equation(self, df=None, equation=None, temp=None):

        def func(x):
            y = eval(equation)
            return y

        if df.shape[0] > 0:

            if equation is not None:

                for col in self.__res:
                    if col in equation:
                        col_name = f"x['{col}']"
                        equation = equation.replace(col, col_name)
                if 'value' in equation:
                    equation = equation.replace('value',"x['value']")

                if 'time_local' in equation:
                    df['time_local'] = pd.to_datetime(df['time_local'])
                    df['time_only'] = df['time_local'].dt.time.astype('str')
                    equation = equation.replace('time_local', 'x["time_only"]')
                    time_list = re.findall(r'\d{2}\:\d{2}\:\d{2}', equation)
                    for tt in time_list:
                        time_split = tt.split(":")
                        updated_time = time(int(time_split[0]), int(time_split[1]), int(time_split[2]))
                        equation = re.sub(tt, str(updated_time), equation)

                df['value'] = df.apply(func, axis=1)

            else:
                df['value'] = df['alias_a']

        return df


    @staticmethod
    def apply_aggregation(self, df=None, aggregation='mean', kpi=None):

        if df.shape[0] > 0:
            if aggregation == 'perc':
                aggregation = 'mean'

            df.set_index('time', inplace=True)

            x = f"df.resample('D').{aggregation}()"

            df = eval(x)

            df.reset_index(inplace=True)

            df['kpi'] = kpi

        df.loc[df["value"] == True, "value"] = 1
        df.loc[df["value"] == False, "value"] = 0

        return df

    @staticmethod
    def __alias_substrings(df, in_list, col_name='alias'):
        """ This function returns a list of aliases for a given reference in the
            the config file.
        Parameters
        ----------
        df: dataframe
        It's the source dataframe
        in_list: list
        It's the list of given aliases in the config file
        col_name: string
        It's the column name in the source dataframe
        Returns
        -------
        filtered_aliases: list
        This list contains the aliases that are filtered out
        """
        groups = in_list

        substrings = []
        filtered_aliases = []
        # Creating list of substrings
        for group_id in groups:
            print("group_id ==> ", group_id)
            print("group_id data type ==> ",type(group_id))
            substrings.append(group_id.split('_'))

        unique_aliases = df.columns
        # print(unique_aliases)
        # Removing None from the list of unique aliases
        unique_aliases = list(filter(None, unique_aliases))
        # Filter the aliases that are to be used for the KPI
        for alias in unique_aliases:
            for sub in substrings:
                if all(a in alias for a in sub):
                    filtered_aliases.append(alias)

        return filtered_aliases

    @staticmethod
    def __batch_subset(batch, start_time, end_time):
        """Gets a dataframe and a specific duration, where it takes a subset
        of dataframe that falls in the start and end times.
        Parameters
        ----------
        batch: dataframe
        The source dataframe
        start_time: timestamp
        This will be the start or minimum time of the dataframe
        end_time: timestamp
        This be the end or maximum time of the dataframe
        Returns
        ------
        batch: dataframe
        This is resultant dataframe between the start and end times
        batch_min_time: timestamp
        The minimum time of the dataframe
        batch_max_time: timestamp
        The maximum time of the dataframe
        """
        # in case start time is given
        if start_time is not None:
            # start_time = str(start_time)
            # split the time(string) in hour,min and secs
            hour = start_time.split(':')[0]
            minute = start_time.split(':')[1]
            second = start_time.split(':')[2]
            print(f'start_time {time(int(hour), int(minute), int(second))}')
            # filter dataframe having time  greater or equal  to start time
            batch = batch.loc[batch['time'].dt.time >= time(int(hour), int(minute), int(second))]
        # in case end time is given
        if end_time is not None:
            # end_time = str(end_time)
            # split the time(string) in hour,min and secs
            hour = end_time.split(':')[0]
            minute = end_time.split(':')[1]
            second = end_time.split(':')[2]
            print(f"end_time {time(int(hour), int(minute), int(second))}")
            # filter dataframe having time  smaller than the  end time
            batch = batch.loc[batch['time'].dt.time < time(int(hour), int(minute), int(second))]
        # get miminium and maximum time of the dataframe
        batch_min_time = batch.time.min()
        batch_max_time = batch.time.max()
        return batch, batch_min_time, batch_max_time

    @staticmethod
    def alias_pairs_dataframe(self, data_rd=None, aliases=None, kpi=None, resample_grain=None,
                              daily_resample=None, agg=None, non_negative=None):
        df_master = pd.DataFrame()
        res = [index for index, val in enumerate(aliases)]
        alias_cols = aliases.copy()
        alias_cols.append('time')
        cols = [val for index, val in enumerate(self.__res) if index in res]
        cols.append('time')
        df = data_rd[alias_cols].copy()
        df.columns = cols
        if df.shape[0] > 0:
            df_master = df
        return df_master

    @staticmethod
    def alias_dataframe(self, data_rd=None, aliases=None, kpi=None, resample_grain=None,
                        daily_resample=None, agg=None, non_negative=None):
        df_master = pd.DataFrame()
        df = data_rd[['time', aliases]].copy()
        df.rename(columns={aliases: 'alias_a'}, inplace=True)
        df_master = df

        return df_master

    @staticmethod
    def __alias_pairs(self, data_rd, rule, aliases):
        #print(data_rd)
        self.__index = [index for index, val in enumerate(aliases)]  #creating a new list i.e list of elements indeces of aliases
        col_names = self.__res   #assign sef.__res to col_names variable
        #print(col_names)
        # col_names list will only contain elements from the original col_names list, where the index of the element 
        #is present in the __index list. enumerate() returns iterator that produces the rows with index
        col_names = [val for index, val in enumerate(col_names) if index in self.__index]
        #print(col_names)
        
        #creating a list that is iterating over a list of aliases, enumerate() returns an iterator that produces the row
        #with index, it then iterates over the rows produced by the enumerate(aliases), it then assisgn the 1st element
        #(index) to the variable index and the 2nd element(value) to the variable val, and then if the legth of the 
        #element ==0, the index of the current element is appended to the null_list_index list.
        null_list_index = [index for index, val in enumerate(aliases) if len(val) == 0]
        print(null_list_index)
        non_null_dfs_list = []  #empty list for non-null dfs
        null_dfs_list = []  #empthy list for null dfs
        

        #iterate over the elements of the aliases
        for i in range(len(aliases)):

            non_null_alias_df = pd.DataFrame(columns=['alias', 'keys']) #df with alias and key column
            null_alias_df = pd.DataFrame(columns=['alias', 'keys']) #df with alias and key column
            #checks if the length of the current element of the aliases list is greater than zero,
            if len(aliases[i]) > 0:
                #it enters another for loop which iterates over the elements of the current element
                for alias in aliases[i]:
                    #It then splits the current element of the inner for loop into a list called keys
                    keys = alias.split("_")
                    #print(keys)
                    
                    #It is likely that the rule list contains some rules or criteria that are used to process the data, and this 
                    #loop is used to remove any elements from keys that are specified in the rules. The variable col_names is a 
                    #subset of __res variable and contains the columns that matched the indexes in the __index variable
                    for ele in rule[col_names[i]].strip().split('_'):
                        keys.remove(ele)

                    
                    
                    
                    #If the length of the keys list is zero, it appends the current alias to the null_alias_df DataFrame
                    if len(keys) == 0:

                        df = pd.DataFrame([[alias, None]], columns=['alias', 'keys'])
                        null_alias_df = pd.concat([null_alias_df, df], axis=0, ignore_index=True)
                        #print(null_alias_df)


                    #it combines the remaining elements of the keys list into a single string and appends the 
                    #current alias and the combined string to the non_null_alias_df DataFrame

                    else:

                        key = '_'.join(map(str, keys))
                        df = pd.DataFrame([[alias, key]], columns=['alias', 'keys'])
                        non_null_alias_df = pd.concat([non_null_alias_df, df], axis=0, ignore_index=True)
                        #print(non_null_alias_df)

            
            #Then it checks the shape of non_null_alias_df and null_alias_df and appends it to the non_null_dfs_list 
            #and null_dfs_list respectively
            if non_null_alias_df.shape[0] > 0:
                non_null_dfs_list.append(non_null_alias_df)
            if null_alias_df.shape[0] > 0:
                null_dfs_list.append(null_alias_df)

        #Then it checks the length of the null_dfs_list and non_null_dfs_list
        if len(null_dfs_list) == len(col_names):
            #nulls_df_merge, contains the Cartesian product of all the DataFrames in the null_dfs_list.
            nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, how="cross"), null_dfs_list)
            
            #It creates a new DataFrame non_nulls_df_merge with columns alias, keys and no rows.
            non_nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])
        else:

            if len(non_null_dfs_list) > 1:
                non_nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, on=["keys"], how="inner"),
                                            non_null_dfs_list)
            elif len(non_null_dfs_list) == 1:
                non_nulls_df_merge = non_null_dfs_list[0]
            else:
                non_nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

            if len(null_dfs_list) > 1:
                nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, how="cross"), null_dfs_list)
            elif len(null_dfs_list) == 1:
                nulls_df_merge = null_dfs_list[0]
            else:
                nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

        #checking the no of rows of nulls_df_merge.shape and non_nulls_df_merge.shape
        if nulls_df_merge.shape[0] > 0 and non_nulls_df_merge.shape[0] > 0:
            alias_pairs_df = pd.merge(nulls_df_merge, non_nulls_df_merge, how='cross')
        elif nulls_df_merge.shape[0] == 0:
            alias_pairs_df = non_nulls_df_merge
        elif non_nulls_df_merge.shape[0] == 0:
            alias_pairs_df = nulls_df_merge
            #print(alias_pairs_df)

        #removing all the columns with names containing the string 'keys' from the alias_pairs_df DataFrame.
        subs = 'keys'
        keys_cols = [col for col in alias_pairs_df if subs in col]
        alias_pairs_df = alias_pairs_df.drop(keys_cols, axis=1)

        if alias_pairs_df.shape[1] == len(col_names):
            alias_pairs = alias_pairs_df.values.tolist()
            print(alias_pairs)
        else:
            alias_pairs = []

        return alias_pairs

    def return_log(self):
        return self.__df_logs

In [138]:
    @staticmethod
    '''
    def __alias_pairs1(self, data_rd, rule, aliases):
        self.__index = [index for index, val in enumerate(aliases)]
        col_names = self.__res
        col_names = [val for index, val in enumerate(col_names) if index in self.__index]
        null_list_index = [index for index, val in enumerate(aliases) if len(val) == 0]
        non_null_dfs_list = []
        null_dfs_list = []

        for i in range(len(aliases)):

            non_null_alias_df = pd.DataFrame(columns=['alias', 'keys'])
            null_alias_df = pd.DataFrame(columns=['alias', 'keys'])
            if len(aliases[i]) > 0:
                for alias in aliases[i]:
                    keys = alias.split("_")
                    for ele in rule[col_names[i]].strip().split('_'):
                        keys.remove(ele)

                    if len(keys) == 0:

                        df = pd.DataFrame([[alias, None]], columns=['alias', 'keys'])

                        null_alias_df = pd.concat([null_alias_df, df], axis=0, ignore_index=True)
                        print(null_alias_df)


                    else:

                        key = '_'.join(map(str, keys))
                        df = pd.DataFrame([[alias, key]], columns=['alias', 'keys'])
                        non_null_alias_df = pd.concat([non_null_alias_df, df], axis=0, ignore_index=True)

            if non_null_alias_df.shape[0] > 0:
                non_null_dfs_list.append(non_null_alias_df)
            if null_alias_df.shape[0] > 0:
                null_dfs_list.append(null_alias_df)

        if len(null_dfs_list) == len(col_names):
            nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, how="cross"), null_dfs_list)
            non_nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])
        else:

            if len(non_null_dfs_list) > 1:
                non_nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, on=["keys"], how="inner"),
                                            non_null_dfs_list)
            elif len(non_null_dfs_list) == 1:
                non_nulls_df_merge = non_null_dfs_list[0]
            else:
                non_nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

            if len(null_dfs_list) > 1:
                nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, how="cross"), null_dfs_list)
            elif len(null_dfs_list) == 1:
                nulls_df_merge = null_dfs_list[0]
            else:
                nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

        if nulls_df_merge.shape[0] > 0 and non_nulls_df_merge.shape[0] > 0:
            alias_pairs_df = pd.merge(nulls_df_merge, non_nulls_df_merge, how='cross')
        elif nulls_df_merge.shape[0] == 0:
            alias_pairs_df = non_nulls_df_merge
        elif non_nulls_df_merge.shape[0] == 0:
            alias_pairs_df = nulls_df_merge

        subs = 'keys'
        keys_cols = [col for col in alias_pairs_df if subs in col]
        alias_pairs_df = alias_pairs_df.drop(keys_cols, axis=1)

        if alias_pairs_df.shape[1] == len(col_names):
            alias_pairs = alias_pairs_df.values.tolist()
        else:
            alias_pairs = []

        return alias_pairs

    def return_log(self):
        return self.__df_logs
    '''
    '''
    with open('new_config.json') as file:
    config = json.loads(file.read())
    file.close()
    station_id = config['settings']['station_id']
    print('station id ==> ',station_id)
    df = pd.DataFrame(config['rules'])
    df
    
    #station_id = config['settings']['station_id']
    #print('station id ==> ',station_id)
    '''

SyntaxError: invalid syntax (3812942811.py, line 2)

In [139]:
def kpi_function(kwargs):
    """ This function actually reads data from the source table and perform operations on it and then
        sends back the resultant logs to the destination table in Bigquery.
        Parameters
        ----------
        start_time: timestamp
        It the time from which data will start
        end_time: timestamp
        It the end of the data
        project: GCP project id
        dataset_src: Bigquery source dataset
        location: Project location
        dataset_dest: Bigquery destination dataset
        table_src: Bigquery source table
        table_dest: Bigquery destination table
        Returns
        -------
        This function doesn't return anything
        -------
    """

    start_time = kwargs['start_time']
    end_time = kwargs['end_time']
    project = kwargs['project']
    client_id = kwargs['client_id']
    device_id = kwargs['device_id']
    dataset_src = kwargs['dataset_src']
    dataset_dest = kwargs['dataset_dest']
    table_src = kwargs['table_src']
    table_dest = kwargs['table_dest']
    location = kwargs['location']
    config = kwargs['config']

    def station_data(station_id, start, end):
        data = Daily(station_id, start, end)
        data = data.fetch()
        data.reset_index(inplace=True)
        data = data[['time', 'tavg', 'tmin', 'tmax']]
        #     print(data)
        return data

    current_time = arrow.utcnow().floor('day')
    # if end_time <= current_time:
    output = config

    rules = pd.DataFrame(output['rules'])
    rules.index = rules.index.astype(int)
    rules.sort_index(inplace=True)
    rules = rules[['kpi_name',
                   'alias_a',
                   'alias_b',
                   'alias_c',
                   'alias_d',
                   'alias_e',
                   'alias_f',
                   'equation_1',
                   'equation_2',
                   'aggregation',
                   'start_time',
                   'end_time',
                   'mute_logs']].copy()
    rules = rules.where(pd.notnull(rules), None)
    settings = output['settings']['mute_rules']
    try:
        station_id = output['settings']['station_id']
    except:
        print(f'Station ID is not present for {client_id} ')
    df = rules

    # create an empty dataframe
    logs = pd.DataFrame()
    # get access to bigqyery
#     credentials, project_id = google.auth.default()
    #client = bigquery.Client()

    if settings is False:

        #dataframe = pd.read_csv('main_data.csv')
        dataframe=pd.read_csv('D:\\Cloud-Track\\assign1\\main_data.csv')
        #print(dataframe)

        data_rd = dataframe
        data_rd['value'] = data_rd['value'].astype('float')
        data_rd['time'] = data_rd['time_local']
        data_rd['time'] = pd.to_datetime(data_rd['time'], format='%Y-%m-%d %H:%M:%S')
        dff2 = data_rd[['time', 'alias','value']].copy()
        dff2 = dff2[dff2.alias == 'outside_air_temp']
        dff2.set_index('time',inplace=True)
        dff2 = dff2.resample('1h').first()

        dff2['alias'] = 'air_temperature_1h'
        dff2.reset_index(inplace=True)

        dff2['time_local'] = dff2['time']

        frames = [data_rd, dff2[['time_local', 'time', 'alias', 'value']]]
        data_rd = pd.concat(frames, axis=0).reset_index(drop=True)
        if data_rd.shape[0] > 0:
            left_table = data_rd.drop(['alias','value','time_local'],axis=1)
            left_table = left_table.groupby("time").min().reset_index()
            sub_df = data_rd[['time','alias','value']].copy()
            sub_df['value'] = pd.to_numeric(sub_df['value'])
            right_table = pd.pivot_table(sub_df,index='time',columns='alias',values='value').reset_index()
            data_rd = pd.merge(left_table,right_table,on='time', how='outer')
            df_data_rd= data_rd
            #print(df_data_rd)

        if data_rd.shape[0] > 0:
            min_time = data_rd.time.min()
            max_time = data_rd.time.max()
            try:
                if station_id is not None:
                    meteo_data = station_data(station_id, min_time, max_time)
                    data_rd = data_rd.merge(meteo_data, on='time', how='left')
                    data_rd[['tavg', 'tmax', 'tmin']] = data_rd[['tavg', 'tmax', 'tmin']].astype('float')
                else:
                    print('Station is None')
            except Exception as e:
                print(e)

            for i in range(df.shape[0]):
                obj = kpi(df.iloc[i], data_rd)
                dff = obj.return_log()
                logs = pd.concat([logs, dff], axis=0)

                if df.iloc[i]['mute_logs']:
                    aliases_list = df.iloc[i]['mute_logs'].split(',')
                    for alias in aliases_list:
                        logs = logs[~logs.aliases.str.contains(alias.strip())]
        else:
            print(f"Data is not available from {start_time} to {end_time}")


        if logs.shape[0]>0:

            logs['time'] = logs['time'].dt.date

            logs.rename(columns={"time":"date"},inplace=True)
            logs['client_id'] = client_id
            logs['device_id'] = device_id
            logs['date'] = logs['date'].astype(str)
            # load the logs dataframe to destination table in the bigquery
            #table = client.get_table(f"{project}.{dataset_dest}.{table_dest}")


        else:
            print('no logs saved')
    else:
        print("All rules are muted")

In [140]:
## read the config file here 
# ======================================
with open('D:\\Cloud-Track\\assign1\\kpi_config.json', 'r') as f:
    config = json.load(f)
    print(config)
    
    rules=config['rules']
    rules=pd.DataFrame(rules)
    #print(rules)
    rules['mute_logs'] = None
    rules=rules[rules.kpi_name == 'run_time_fraction']
    print(rules)

{'_comment_': 'This is KPI settings', 'settings': {'mute_rules': False, 'station_id': '10567'}, 'rules': {'kpi_name': {'1': 'run_time_hours', '2': 'run_time_fraction', '3': 'ahu_air_return_temp_fraction_18C-24C', '4': 'hx_1_hot_water_delta_temp_sum', '5': 'hx_2_hot_water_delta_temp_sum', '6': 'outside_air_temp_mean', '7': 'outside_air_temp_max', '8': 'HDD_15C', '9': 'CDD_15C', '10': 'co2_ppm_fraction_<500ppm', '11': 'co2_ppm_mean', '12': 'ahu_air_return_temp_mean', '13': 'hot_water_temp_mean', '14': 'predicted_heat_kwh', '15': 'baseline_gas_kwh'}, 'alias_a': {'1': 'enable_cmd', '2': 'enable_cmd', '3': 'ahu_air_return_temp_sensor', '4': 'heatExchanger_1_primaryLoop_hot_water_leaving_temp_sensor', '5': 'heatExchanger_2_primaryLoop_hot_water_leaving_temp_sensor', '6': 'air_temperature_1h', '7': 'air_temperature_1h', '8': 'air_temperature_1h', '9': 'air_temperature_1h', '10': 'ahu_co2_return_sensor', '11': 'ahu_co2_return_sensor', '12': 'ahu_air_return_temp_sensor', '13': 'heatExchanger_se

In [144]:
with open('D:\\Cloud-Track\\assign1\\kpi_config.json') as file:
    config = json.loads(file.read())
    file.close()
df = pd.DataFrame(config['rules'])
#print(df)
df
rules= rules[rules.kpi_name =='outside_air_temp_mean']
#rules

In [142]:
# data for not more than 5 days
start_time = arrow.get('2022-10-01 00:00:00')
end_time  = arrow.get('2022-12-31 00:00:00')

# You should not edit kwargs, except your config variable
kwargs = {
      'start_time'      : start_time,
      'end_time'        : end_time,
      'project'         : 'thermosphr-prod',
      'client_id'       : 'gera-arcaden-germany',
      'dataset_src'     : 'bi',
      'location'        : 'Europe/Berlin',
      'dataset_dest'    : 'bi',
      'table_src'       : 'datamart_v2',
      'table_dest'      : 'bi',
      'config'          : config,                      # your config should be given here.
      'device_id'       : 'ayQAWQgc'
}

kpi_function(kwargs)

column name ==>  alias_a
group_id ==>  enable_cmd
group_id data type ==>  <class 'str'>
column name ==>  alias_a
group_id ==>  enable_cmd
group_id data type ==>  <class 'str'>
column name ==>  alias_a
group_id ==>  ahu_air_return_temp_sensor
group_id data type ==>  <class 'str'>
column name ==>  alias_b
group_id ==>  ahu_fan_air_enable_return_cmd
group_id data type ==>  <class 'str'>
[]
[['ahu_10_air_return_temp_sensor', 'ahu_10_fan_air_return_enable_cmd'], ['ahu_11_air_return_temp_sensor', 'ahu_11_fan_air_return_enable_cmd'], ['ahu_12_air_return_temp_sensor', 'ahu_12_fan_air_return_enable_cmd'], ['ahu_1_air_return_temp_sensor', 'ahu_1_fan_air_return_enable_cmd'], ['ahu_2_air_return_temp_sensor', 'ahu_2_fan_air_return_enable_cmd'], ['ahu_3_air_return_temp_sensor', 'ahu_3_fan_air_return_enable_cmd'], ['ahu_4_air_return_temp_sensor', 'ahu_4_fan_air_return_enable_cmd'], ['ahu_5_air_return_temp_sensor', 'ahu_5_fan_air_return_enable_cmd'], ['ahu_6_air_return_temp_sensor', 'ahu_6_fan_air_ret